In [ ]:
!pip install skforecast
!pip install dask[dataframe]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from joblib import load
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
from skforecast.plot import set_dark_theme
from skforecast.preprocessing import exog_long_to_dict

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/Machine Learning - TF/Modelo_SKForecast/df_skforecast.csv')

df_actual = pd.read_csv('/content/drive/MyDrive/Datasets/Machine Learning - TF/InformationByTokenClass.csv')

forecaster = load('/content/drive/MyDrive/Datasets/Machine Learning - TF/Modelo_SKForecast/skforecaster_model.joblib')

In [ ]:
df['last_updated'] = pd.to_datetime(df['last_updated'])

In [ ]:
# Filtrar los tokens que están en el dataset actual
tokens_actuales = df_actual['Name'].unique()
df_filtrado = df[df['name'].isin(tokens_actuales)]

In [ ]:
# Obtener el último registro por token
latest_data = df_filtrado.sort_values(by='last_updated').groupby('name').last()

# Filtrar tokens con marketCap menor a 1 millón y mayor a 100
low_cap_tokens = latest_data[(latest_data['marketCap'] < 1e6) & (latest_data['marketCap'] > 100000)]

# Obtener los nombres de los tokens de baja capitalización
low_cap_token_names = low_cap_tokens.index.tolist()

print(len(low_cap_token_names))
print("Tokens de baja capitalización:", low_cap_token_names)

36
Tokens de baja capitalización: ['ASPO World', 'Acquire.Fi', 'Aimedis (new)', 'Binamon', 'Binemon', 'Blockchain Monster Hunt', 'Chain of Legends', 'ChainGuardians', 'CryptoBlades', 'DOGGY', 'DOSE', 'Dawn Protocol', 'Defina Finance', 'Dotmoovs', 'Fabwelt', 'Forest Knight', 'Garlicoin', 'GuildFi', 'Heroes Chained', 'Kaby Arena', 'Loser Coin', 'Mist', 'My Master War', 'Ninneko', 'Playermon', 'Raze Network', 'RealFevr', 'SHILL Token', 'Seascape Crowns', 'TRAVA.FINANCE', 'VisionGame', 'WOOF', 'X World Games', 'XCarnival', 'XMON', 'aiRight']


In [ ]:
names = unique_names = df['name'].unique().tolist()

In [ ]:
for name in names:
  print(name)

0x Protocol
1MillionNFTs
2crazyNFT
ACA Token
ACoconut
ADAPad
AIPAD
ALL BEST ICO
AME Chain
APENFT
APYSwap
ASKO
ASPO World
ASTA
ATOR Protocol
AXEL
AcknoLedger
Acquire.Fi
Across Protocol
AdEx
AdaSwap
Aergo
AhaToken
Aimedis (new)
AirDAO
AirNFTs
Akash Network
Alanyaspor Fan Token
Alfa Romeo Racing ORLEN Fan Token
Alien Worlds
Alitas
AllianceBlock
AllianceBlock Nexera
Alpaca Finance
Alpha Quark Token
Alphr finance
Alpine F1 Team Fan Token
Aluna.Social
Amazy
Amp
Anchor Protocol
Ancient Raid
Angola
AnimalGo
AntiMatter Token
ApeCoin
Apron Network
ArGo
Archimedes Finance
Ardor
Ares Protocol
Argentine Football Association Fan Token
Argon
Ariva
Arweave
Aston Martin Cognizant Fan Token
Aston Villa Fan Token
Astrafer
AstroSwap
Atmos
AtromG8
Aura Finance
Aurigami
Aurox
AurusX
Auto
Autobahn Network
Avalanche
Axie Infinity
Axis DeFi
BENQI
BHO Network
BSCStation
Baanx
BabySwap
Badger DAO
Balancer
Battle Hero
Battle World
Beam
Beer Money
BendDAO
Berry Data
Beta Finance
Beyond Protocol
BiFi
Big Data Proto

In [ ]:
def predict(name, steps_p, df):
    # Filtrar los datos del token específico
    df_pred = df[df['name'] == name]
    df_pred = df_pred.sort_values(by='last_updated')

    # Preparar la serie y las variables exógenas
    series = df_pred[['name', 'last_updated', 'marketCap']]
    exog = df_pred[['name', 'last_updated', 'cmcRank', 'price', 'volume24h', 'percentChange1h', 'percentChange24h']]

    # Convertir las variables exógenas a diccionario para el modelo
    exog_dict = exog_long_to_dict(
        data=exog,
        series_id='name',
        index='last_updated',
        freq='W'  # Frecuencia semanal
    )

    # Realizar la predicción para los próximos 'steps_p' periodos
    predicciones = forecaster.predict(steps=steps_p, exog=exog_dict, suppress_warnings=True)
    predicciones.index = pd.date_range(start=df_pred['last_updated'].max(), periods=steps_p, freq='W')

    # Filtrar las predicciones solo para el token seleccionado
    if name in predicciones.columns:
        predicciones = predicciones[[name]]
    else:
        raise ValueError(f"No se encontraron predicciones para el token: {name}")

    # Crear el gráfico interactivo
    fig = go.Figure()

    # Agregar la línea del marketCap original
    fig.add_trace(go.Scatter(
        x=series['last_updated'],
        y=series['marketCap'],
        mode='lines',
        name='MarketCap Observado',
        line=dict(color='blue')
    ))

    # Agregar la línea de las predicciones
    fig.add_trace(go.Scatter(
        x=predicciones.index,  # Fechas de las predicciones
        y=predicciones[name].values,  # Valores predichos para la serie
        mode='lines',
        name='MarketCap Predicho',
        line=dict(color='orange', dash='dot')
    ))

    # Personalizar el diseño del gráfico
    fig.update_layout(
        title=f'Predicción del MarketCap para {name}',
        xaxis_title='Fecha',
        yaxis_title='MarketCap',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        template='plotly_white'
    )

    # Devolver las predicciones
    return fig, predicciones

In [ ]:
fig, pred = predict(name='TheForce Trade', steps_p=20, df=df)
fig.show()

/usr/local/lib/python3.10/dist-packages/skforecast/preprocessing/preprocessing.py:494: MissingValuesWarning:

Exogenous variables for series 'TheForce Trade' are incomplete. NaNs have been introduced after setting the frequency. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)



In [ ]:
pred

,Æternity
2024-11-03,-4.828230e+08
2024-11-10,-4.828230e+08
2024-11-17,-4.828230e+08
2024-11-24,-4.828230e+08
2024-12-01,-4.828230e+08
2024-12-08,-4.828230e+08
2024-12-15,-4.828230e+08
2024-12-22,-4.828230e+08
2024-12-29,-4.828230e+08
2025-01-05,-4.828230e+08


In [ ]:
def get_top_performers(tokens_list, df, steps_p=10):
    performance = []

    # Iterar sobre los tokens filtrados
    for token in tokens_list:
        # Llamar a la función predict para obtener las predicciones para cada token
        fig, predicciones = predict(name=token, steps_p=steps_p, df=df)

        # Verificar que las predicciones estén disponibles
        if not predicciones.empty:
            # Obtener el valor inicial de marketCap (última fecha observada)
            initial_value = df[df['name'] == token].iloc[-1]['marketCap']

            # Obtener los valores predichos para los próximos 10 días
            predicted_values = predicciones[token].values

            # Calcular la subida total (porcentaje de cambio)
            max_predicted_value = predicted_values.max()  # Valor máximo predicho
            min_predicted_value = predicted_values.min()  # Valor mínimo predicho

            # Calcular la subida porcentual de los valores predichos
            increase_percentage = (max_predicted_value - min_predicted_value) / min_predicted_value * 100

            # Almacenar el resultado
            performance.append({'token': token, 'increase_percentage': increase_percentage, 'max_predicted_value': max_predicted_value})

    # Convertir la lista de resultados en un DataFrame
    performance_df = pd.DataFrame(performance)

    # Ordenar los tokens por el porcentaje de aumento (de mayor a menor)
    top_performers = performance_df.sort_values(by='increase_percentage', ascending=False)

    return top_performers

In [ ]:
top_tokens = get_top_performers(tokens_list=low_cap_token_names, df=df, steps_p=10)

/usr/local/lib/python3.10/dist-packages/skforecast/preprocessing/preprocessing.py:494: MissingValuesWarning:

Exogenous variables for series 'ASPO World' are incomplete. NaNs have been introduced after setting the frequency. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)

/usr/local/lib/python3.10/dist-packages/skforecast/preprocessing/preprocessing.py:494: MissingValuesWarning:

Exogenous variables for series 'Acquire.Fi' are incomplete. NaNs have been introduced after setting the frequency. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)

/usr/local/lib/python3.10/dist-packages/skforecast/preprocessing/preprocessing.py:494: MissingValuesWarning:

Exogenous variables for series 'Binamon' are incomplete. NaNs have been introduced after setting the frequency. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)

/usr/local/lib/pyth

In [ ]:
top_tokens

,token,increase_percentage,max_predicted_value
17,GuildFi,201.313031,1.827614e+07
30,VisionGame,138.543975,8.348807e+06
18,Heroes Chained,101.986767,8.348807e+06
27,SHILL Token,92.176693,7.943323e+06
22,My Master War,80.412111,8.348807e+06
2,Aimedis (new),68.432457,7.943323e+06
8,CryptoBlades,52.991164,8.310571e+06
6,Chain of Legends,52.319553,8.348807e+06
7,ChainGuardians,52.319553,8.348807e+06
12,Defina Finance,52.319553,8.348807e+06


In [ ]:
top_tokens.to_csv('top_tokens.csv', index=False)